## Veckodagar, avlidna

Jag ska undersöka om det finns något att upptäcka om man grupperar dödsfallen per veckodag.

Jag börjar med att läsa in bladet "Antal avlidna per dag" per dag och plocka bort sista raden som är okända. Kolumnen "Datum" lästes inte in som ett datum så jag fick köra en .to_datetime().

In [23]:
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Antal avlidna per dag", names = {"Datum", "Avlidna"}).iloc[0:-1]

df["Datum"] = pd.to_datetime(df["Datum"])

df.tail()

,Datum,Avlidna
585,2021-10-17,2
586,2021-10-18,1
587,2021-10-19,1
588,2021-10-20,1
589,2021-10-21,0


Först konverterar jag datumen till numeriska veckodagar och gör en ny dataframe av den vecko-grupperade och summerade ursprungliga dataframen.

In [24]:
from datetime import datetime

df["Veckodag"] = df["Datum"].dt.dayofweek

df_per_veckodag = df.groupby("Veckodag")[["Avlidna"]].sum().reset_index()

df_per_veckodag

,Veckodag,Avlidna
0,0,2142
1,1,2055
2,2,2234
3,3,2273
4,4,2153
5,5,2022
6,6,2077


In [25]:
import plotly_express as px

# Så här kan vi göra eftersom dataframen är sorterad efter de ursprungliga numeriska veckodagarna
df_per_veckodag["Veckodag"] = ["Måndag", "Tisdag", "Onsdag", "Torsdag", "Fredag", "Lördag", "Söndag"]

fig_px = px.bar(df_per_veckodag, x = "Veckodag", y = "Avlidna", title = "Avlidna per veckodag")

fig_px.write_html(f"./Visualiseringar/avlidna_per_veckodag.html")
fig_px.show()


I resultet ser man inte mycket intressant tycker jag. Skillnaderna är för små för att dra några slutsatser. De skillnader som ändå finns kanske kan förklaras med varierande arbetsbelastning för den som registerar dödsfallen kombinerat med en osäkerhet när personen avled. Detta är givetvis bara gissningar.

## Antal fall regioner över tid

Nu tänkte jag att man kunde titta på hur det såg ut mellan regionerna över tid. Excellbladet "Antal per dag region" ser intressant ut.

In [26]:
# Antal per dag / region, plotta skillnaderna
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Antal per dag region")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626 entries, 0 to 625
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Statistikdatum       626 non-null    datetime64[ns]
 1   Totalt_antal_fall    626 non-null    int64         
 2   Blekinge             626 non-null    int64         
 3   Dalarna              626 non-null    int64         
 4   Gotland              626 non-null    int64         
 5   Gävleborg            626 non-null    int64         
 6   Halland              626 non-null    int64         
 7   Jämtland_Härjedalen  626 non-null    int64         
 8   Jönköping            626 non-null    int64         
 9   Kalmar               626 non-null    int64         
 10  Kronoberg            626 non-null    int64         
 11  Norrbotten           626 non-null    int64         
 12  Skåne                626 non-null    int64         
 13  Stockholm            626 non-null  

Okej, det ser ju fint ut...

In [28]:
df_antal_dag_region = df.copy()

# Skapar en lista över regionerna från columnerna för att ha till y-axeln när jag plottar
columns = [column for column in df_antal_dag_region.columns][2:]

# Skapar ett rullande medelvärde över 7 dagar och droppar de rader i början som inte fått några värden
df_antal_dag_region[columns] = df_antal_dag_region[columns].rolling(7).mean().dropna()

fig_px = px.line(df_antal_dag_region, x = "Statistikdatum", y = columns)
fig_px.write_html(f"./Visualiseringar/antal_per_dag_region.html")
fig_px.show()

Statistiken var väldigt hackig utan medelvärdet, troligtvis beror det på att ingen statistik fördes in på helgerna.

Med det rullande medelvärdet ser det bättre ut, men det är ganska rörigt och något speciellt framkommer inte. Det verkar dock som att Stockholm nästan alltid går in i "vågorna" ett par veckor tidigare än övriga regioner.

## Antal fall per capita och region

För att justera bort den stora skillnaden i föregående "analys" ska jag undersöka hur det ser ut vecka för vecka per 100 000 invåndare i regionerna.

Blad "Veckodata Region", således...

In [29]:
import pandas as pd

covid_file = "Folkhalsomyndigheten_Covid19.xlsx"

df = pd.read_excel(covid_file, sheet_name = "Veckodata Region")

df["Vecka"] = df["år"].astype(str) + "v" + df["veckonummer"].astype(str)

df.head()

,år,veckonummer,Region,Antal_fall_vecka,Kum_antal_fall,Antal_intensivvårdade_vecka,Kum_antal_intensivvårdade,Antal_avlidna_vecka,Kum_antal_avlidna,Antal_fall_100000inv_vecka,Kum_fall_100000inv,Vecka
0,2020,1,Blekinge,0,0,0,0,0,0,0,0,2020v1
1,2020,2,Blekinge,0,0,0,0,0,0,0,0,2020v2
2,2020,3,Blekinge,0,0,0,0,0,0,0,0,2020v3
3,2020,4,Blekinge,0,0,0,0,0,0,0,0,2020v4
4,2020,5,Blekinge,0,0,0,0,0,0,0,0,2020v5


In [30]:
df_veckodata_region = df[["Vecka", "Region", "Antal_fall_100000inv_vecka"]]

fig_px = px.line(df_veckodata_region, x = "Vecka", y = "Antal_fall_100000inv_vecka", color = "Region")
fig_px.write_html(f"./Visualiseringar/antal_per100000_vecka_region.html")
fig_px.show()

När man justerar för folkmängd utmärker sig inte längre Stockholm. Det är svårt att säga att någon region är sämre än någon annan. En intressant detalj dock är Gotlands extrema topp 2021v31, märkligt nog bara en-två veckor efter Stockholmsveckan i Visby.

## Vaccination över tid per region

In [64]:
import pandas as pd
import plotly_express as px

vaccine_file = "Folkhalsomyndigheten_Covid19_Vaccine.xlsx"

df = pd.read_excel(vaccine_file, sheet_name = "Vaccinerade tidsserie")

df.head()

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
0,52,2020,| Sverige |,2472,0.000289,Minst 1 dos
1,52,2020,| Sverige |,0,0.000000,Färdigvaccinerade
2,52,2020,Stockholm,288,0.000148,Minst 1 dos
3,52,2020,Stockholm,0,0.000000,Färdigvaccinerade
4,52,2020,Uppsala,8,0.000025,Minst 1 dos


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892 entries, 0 to 1891
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Vecka               1892 non-null   int64  
 1   År                  1892 non-null   int64  
 2   Region              1892 non-null   object 
 3   Antal vaccinerade   1892 non-null   int64  
 4   Andel vaccinerade   1892 non-null   float64
 5   Vaccinationsstatus  1892 non-null   object 
dtypes: float64(1), int64(3), object(2)
memory usage: 88.8+ KB


In [66]:
df_vaccinerade_region = df.copy()

# Kör den här gamla godingen, tycker det var ett smidigt sätt att få både år och vecka på samma axel...
df_vaccinerade_region["Vecka"] = df_vaccinerade_region["År"].apply(str) + "v" + df_vaccinerade_region["Vecka"].apply(str).str.zfill(2)

In [67]:
df_vaccinerade_region

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
0,2020v52,2020,| Sverige |,2472,0.000289,Minst 1 dos
1,2020v52,2020,| Sverige |,0,0.000000,Färdigvaccinerade
2,2020v52,2020,Stockholm,288,0.000148,Minst 1 dos
3,2020v52,2020,Stockholm,0,0.000000,Färdigvaccinerade
4,2020v52,2020,Uppsala,8,0.000025,Minst 1 dos
...,...,...,...,...,...,...
1887,2021v41,2021,Jämtland,90406,0.829771,Färdigvaccinerade
1888,2021v41,2021,Västerbotten,206223,0.907891,Minst 1 dos
1889,2021v41,2021,Västerbotten,196346,0.864408,Färdigvaccinerade
1890,2021v41,2021,Norrbotten,188604,0.893312,Minst 1 dos


In [68]:
fig_px = px.line(df_vaccinerade_region[df_vaccinerade_region["Vaccinationsstatus"].isin(["Färdigvaccinerade"])], x = "Vecka", y = "Andel vaccinerade", color = "Region")
fig_px.write_html(f"./Visualiseringar/andel_vaccinerade_vecka_region.html")
fig_px.show()

Det var ju kanske inte en lineplot som fick mig att flyga ur stolen precis. Det hade varit intressant att veta vad som hände i Värmland runt 2020v20, de drar iväg rejält och har en klar ledning hela försommaren och även under hösten.

## Vaccinerade vs avlidna

Med vetskapen om vaccinationsnivå med oss i bagaget kanske det kan vara intressant att se om det kan tänkas ha någon påverkan på antalet avlidna. Lägst andel vaccinerade verkar Stockholm ha medan Värmland har högst.

In [117]:
df_vvss = df_vaccinerade_region[(df_vaccinerade_region["Vaccinationsstatus"].isin(["Färdigvaccinerade"])) & (df_vaccinerade_region["Region"].isin(["Stockholm", "Värmland"]))].copy()

In [97]:
df_vvss

,Vecka,År,Region,Antal vaccinerade,Andel vaccinerade,Vaccinationsstatus
3,2020v52,2020,Stockholm,0,0.000000,Färdigvaccinerade
27,2020v52,2020,Värmland,0,0.000000,Färdigvaccinerade
47,2020v53,2020,Stockholm,0,0.000000,Färdigvaccinerade
71,2020v53,2020,Värmland,0,0.000000,Färdigvaccinerade
91,2021v01,2021,Stockholm,0,0.000000,Färdigvaccinerade
...,...,...,...,...,...,...
1787,2021v39,2021,Värmland,203852,0.861316,Färdigvaccinerade
1807,2021v40,2021,Stockholm,1435025,0.735433,Färdigvaccinerade
1831,2021v40,2021,Värmland,204870,0.865617,Färdigvaccinerade
1851,2021v41,2021,Stockholm,1459095,0.747768,Färdigvaccinerade


In [125]:
stockholm_befolkning = 2402609 / 1000000
varmland_befolkning = 282911 / 1000000

df = pd.read_excel(covid_file, sheet_name = "Veckodata Region")

# Kör den gamla fina veckoserien
df["Vecka"] = df["år"].apply(str) + "v" + df["veckonummer"].apply(str).str.zfill(2)

# Plocka ut färdigvaccinerade och regionerna Stockholm och Värmland från tidigare dataset
df_vvss = df_vaccinerade_region[(df_vaccinerade_region["Vaccinationsstatus"].isin(["Färdigvaccinerade"])) & (df_vaccinerade_region["Region"].isin(["Stockholm", "Värmland"]))].copy()

# Gör om till procent för att lättare kunna jämföra mot avlidna per miljon invånare
df_vvss["Andel vaccinerade"] = df_vvss["Andel vaccinerade"] * 100
df_vvss["Region"] = df_vvss["Region"] + ", vaccinerade"

df_avlidna = df[["Vecka", "Region", "Antal_avlidna_vecka"]].copy()

# Plocka ut Stockholm och Värmland ur tidigare dataset
df_avlidna = df_avlidna[df["Region"].isin(["Stockholm", "Värmland"])].copy()

# Gör om till avlidna per vecka per miljon invånare
df_avlidna.loc[df_avlidna["Region"] == "Stockholm", "Antal_avlidna_vecka"] = df_avlidna.loc[df_avlidna["Region"] == "Stockholm", "Antal_avlidna_vecka"] / stockholm_befolkning
df_avlidna.loc[df_avlidna["Region"] == "Värmland", "Antal_avlidna_vecka"] = df_avlidna.loc[df_avlidna["Region"] == "Värmland", "Antal_avlidna_vecka"] / varmland_befolkning
df_avlidna["Region"] = df_avlidna["Region"] + ", avlidna per 1 milj."

df_avlidna = df_avlidna.rename(columns = {"Antal_avlidna_vecka": "Avlidna per 1M inv"})


fig_px = px.line(
    pd.concat([df_avlidna, df_vvss]), 
    x = "Vecka", 
    y = ["Avlidna per 1M inv", "Andel vaccinerade"], 
    color = "Region", 
    title = "Andel vaccinerade i procent och avlidna per 1M invånare",
    labels = {"value": "Andel"}
)

fig_px.write_html(f"./Visualiseringar/andel_vaccinerade_avlidna.html")
fig_px.show()


Man ser ju hyfsat tydligt att när vaccineringen börjar öka från noll så sjunker dödligheten ganska markand. Tyvärr är Värmland kanske inte den bästa regionen att jämföra mot eftersom befolkningen är så liten, det blir stora och snabba fluktuationer och lite svårtolkat. Dessvärre gör blir det nästan omöjligt att avgöra om större andel vaccinerade leder till färre döda. 

Det har kanske att göra med vilka som vaccineras:

In [135]:
df = pd.read_excel(vaccine_file, "Vaccinerade ålder")

df_andel = df[(df["Region"].isin(["Stockholm", "Värmland"])) & (df["Vaccinationsstatus"] == "Färdigvaccinerade")][["Region", "Åldersgrupp", "Andel vaccinerade"]].copy()

fig_px = px.bar(df_andel, x = "Åldersgrupp", y = "Andel vaccinerade", color = "Region", barmode = "group")
fig_px.write_html(f"./Visualiseringar/andel_vaccinerade_aldersgrupp.html")
fig_px.show()

Skillnaden mellan Stockholm och Värmland är ju inte så stor i de högre åldrarna.